In [1]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
df1 = pd.read_csv(r"D:\desktop\B\数据\标准化后数据\df1.csv")
df2 = pd.read_excel(r"D:\desktop\B\数据\标准化后数据\df2.xlsx")
df1[['mort', 'CONS','MV','CRRT']] = df1[['mort',  'CONS','MV','CRRT']].astype('category')
df2[['mort',  'CONS','MV','CRRT']] = df2[['mort',  'CONS','MV','CRRT']].astype('category')
selected_features = ["CONS", "LDH", "MV", "AST", "CRRT", "U", "L", "HR", "D.Dimer", "CR", "age", "ALT"]
target = 'mort'
X = df1[selected_features]
y = df1[target]
X_train, X_internal_test, y_train, y_internal_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_external_test = df2[selected_features]
y_external_test = df2[target]

In [2]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_metric_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound

# 定义 VotingClassifier 模型
estimators = [
    ('lr', LogisticRegression(random_state=RANDOM_SEED)),
    ('dt', DecisionTreeClassifier(random_state=RANDOM_SEED)),
    ('gnb', GaussianNB())
]

voting_classifier = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)

param_grid = {
    'lr__C': [0.1, 1.0, 10.0],
    'dt__max_depth': [None, 10, 20]
}

grid_voting = GridSearchCV(estimator=voting_classifier, param_grid=param_grid, cv=10, scoring='roc_auc', n_jobs=-1)

best_models_voting = {}

# 逐步添加特征并训练模型
for i in range(1, len(selected_features) + 1):
    current_features = selected_features[:i]
    X_train_current = X_train[current_features]
    X_internal_test_current = X_internal_test[current_features]
    X_external_test_current = X_external_test[current_features]

    grid_voting.fit(X_train_current, y_train)
    best_model_voting = grid_voting.best_estimator_

    # 使用交叉验证在训练集和内部验证集上进行预测
    y_train_pred_cv = cross_val_predict(best_model_voting, X_train_current, y_train, cv=10, method='predict')
    y_train_proba_cv = cross_val_predict(best_model_voting, X_train_current, y_train, cv=10, method='predict_proba')[:, 1]
    
    y_internal_pred_cv = cross_val_predict(best_model_voting, X_internal_test_current, y_internal_test, cv=10, method='predict')
    y_internal_proba_cv = cross_val_predict(best_model_voting, X_internal_test_current, y_internal_test, cv=10, method='predict_proba')[:, 1]

    # 进行外部测试集预测
    y_external_pred_voting = best_model_voting.predict(X_external_test_current)
    y_external_proba_voting = best_model_voting.predict_proba(X_external_test_current)[:, 1]
    
    # 计算训练集的各项指标及其置信区间
    train_metrics = {
        'accuracy': accuracy_score(y_train, y_train_pred_cv),
        'precision': precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0),
        'recall': recall_score(y_train, y_train_pred_cv, average='weighted'),
        'f1_score': f1_score(y_train, y_train_pred_cv, average='weighted'),
        'auc': roc_auc_score(y_train, y_train_proba_cv)
    }

    train_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_train, y_train_proba_cv, roc_auc_score)
    }

    # 计算内部验证集的各项指标及其置信区间
    internal_metrics = {
        'accuracy': accuracy_score(y_internal_test, y_internal_pred_cv),
        'precision': precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0),
        'recall': recall_score(y_internal_test, y_internal_pred_cv, average='weighted'),
        'f1_score': f1_score(y_internal_test, y_internal_pred_cv, average='weighted'),
        'auc': roc_auc_score(y_internal_test, y_internal_proba_cv)
    }

    internal_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_internal_test, y_internal_proba_cv, roc_auc_score)
    }

    # 计算外部验证集的各项指标及其置信区间
    external_metrics = {
        'accuracy': accuracy_score(y_external_test, y_external_pred_voting),
        'precision': precision_score(y_external_test, y_external_pred_voting, average='weighted', zero_division=0),
        'recall': recall_score(y_external_test, y_external_pred_voting, average='weighted'),
        'f1_score': f1_score(y_external_test, y_external_pred_voting, average='weighted'),
        'auc': roc_auc_score(y_external_test, y_external_proba_voting)
    }

    external_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_external_test, y_external_pred_voting, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_external_test, y_external_pred_voting, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_external_test, y_external_pred_voting, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_external_test, y_external_pred_voting, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_external_test, y_external_proba_voting, roc_auc_score)
    }

    best_models_voting[i] = {
        'features': current_features,
        'train_metrics': train_metrics,
        'train_ci': train_ci,
        'internal_metrics': internal_metrics,
        'internal_ci': internal_ci,
        'external_metrics': external_metrics,
        'external_ci': external_ci,
        'best_params': grid_voting.best_params_
    }

# 输出每个模型的结果
for i, results in best_models_voting.items():
    print(f"\nModel with top {i} features: {results['features']}")
    print("Best Parameters:", results['best_params'])
    
    print("Training Set Metrics:")
    for metric, value in results['train_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}")

    print("Internal Validation Set Metrics:")
    for metric, value in results['internal_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}")
    
    print("External Validation Set Metrics:")
    for metric, value in results['external_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}")



Model with top 1 features: ['CONS']
Best Parameters: {'dt__max_depth': None, 'lr__C': 0.1}
Training Set Metrics:
Accuracy: 0.8676 置信区间: 0.8186-0.9118
Precision: 0.8760 置信区间: 0.8312-0.9185
Recall: 0.8676 置信区间: 0.8186-0.9118
F1_score: 0.8706 置信区间: 0.8235-0.9134
Auc: 0.7913 置信区间: 0.7021-0.8720
Internal Validation Set Metrics:
Accuracy: 0.6818 置信区间: 0.5909-0.7727
Precision: 0.6268 置信区间: 0.5063-0.7541
Recall: 0.6818 置信区间: 0.5909-0.7727
F1_score: 0.6424 置信区间: 0.5257-0.7546
Auc: 0.6719 置信区间: 0.5297-0.8038
External Validation Set Metrics:
Accuracy: 0.8558 置信区间: 0.7885-0.9231
Precision: 0.8503 置信区间: 0.7767-0.9215
Recall: 0.8558 置信区间: 0.7885-0.9231
F1_score: 0.8476 置信区间: 0.7696-0.9183
Auc: 0.7628 置信区间: 0.6643-0.8610

Model with top 2 features: ['CONS', 'LDH']
Best Parameters: {'dt__max_depth': None, 'lr__C': 0.1}
Training Set Metrics:
Accuracy: 0.8824 置信区间: 0.8382-0.9265
Precision: 0.8808 置信区间: 0.8342-0.9255
Recall: 0.8824 置信区间: 0.8382-0.9265
F1_score: 0.8815 置信区间: 0.8346-0.9250
Auc: 0.8652 置信区

In [3]:
import pandas as pd
import os

# 设置输出文件夹路径
output_folder = r"D:\desktop\B\结果\表格\内部外部验证结果"
os.makedirs(output_folder, exist_ok=True)  # 确保输出文件夹存在

# 用于存储结果的列表
output_data = []

# 遍历模型结果并将其组织为字典
for i, results in best_models_voting.items():
    model_name = f"Model with top {i} features"
    for metric in results['train_metrics'].keys():
        output_data.append({
            'Model': model_name,
            'Dataset': 'Training Set',
            'Metric': metric.capitalize(),
            'Value': results['train_metrics'][metric],
            'Confidence Interval': f"{results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'Internal Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['internal_metrics'][metric],
            'Confidence Interval': f"{results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'External Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['external_metrics'][metric],
            'Confidence Interval': f"{results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}"
        })

# 将列表转换为 DataFrame
output_df = pd.DataFrame(output_data)

# 设置文件名
output_file = os.path.join(output_folder, "Voting_Results.xlsx")

# 将 DataFrame 保存为 Excel 文件
output_df.to_excel(output_file, index=False)

print(f"结果已保存至：{output_file}")


结果已保存至：D:\desktop\B\结果\表格\内部外部验证结果\Voting_Results.xlsx


7特征

In [4]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_metric_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound

# 使用选定的7个最佳特征
selected_features = ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']
X_train_current = X_train[selected_features]
X_internal_test_current = X_internal_test[selected_features]
X_external_test_current = X_external_test[selected_features]

# 定义 VotingClassifier 模型
estimators = [
    ('lr', LogisticRegression(random_state=RANDOM_SEED)),
    ('dt', DecisionTreeClassifier(random_state=RANDOM_SEED)),
    ('gnb', GaussianNB())
]

voting_classifier = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)

param_grid = {
    'lr__C': [0.1, 1.0, 10.0],
    'dt__max_depth': [None, 10, 20]
}

grid_voting = GridSearchCV(estimator=voting_classifier, param_grid=param_grid, cv=10, scoring='roc_auc', n_jobs=-1)
grid_voting.fit(X_train_current, y_train)
best_model_voting = grid_voting.best_estimator_

# 使用交叉验证在训练集上进行预测
y_train_pred_cv = cross_val_predict(best_model_voting, X_train_current, y_train, cv=10, method='predict')
y_train_proba_cv = cross_val_predict(best_model_voting, X_train_current, y_train, cv=10, method='predict_proba')[:, 1]

# 进行内部验证集预测
y_internal_pred_cv = cross_val_predict(best_model_voting, X_internal_test_current, y_internal_test, cv=10, method='predict')
y_internal_proba_cv = cross_val_predict(best_model_voting, X_internal_test_current, y_internal_test, cv=10, method='predict_proba')[:, 1]

# 进行外部测试集预测
y_external_pred_voting = best_model_voting.predict(X_external_test_current)
y_external_proba_voting = best_model_voting.predict_proba(X_external_test_current)[:, 1]

# 计算训练集的各项指标及其置信区间
train_metrics = {
    'accuracy': accuracy_score(y_train, y_train_pred_cv),
    'precision': precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0),
    'recall': recall_score(y_train, y_train_pred_cv, average='weighted'),
    'f1_score': f1_score(y_train, y_train_pred_cv, average='weighted'),
    'auc': roc_auc_score(y_train, y_train_proba_cv)
}

train_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_train, y_train_proba_cv, roc_auc_score)
}

# 计算内部验证集的各项指标及其置信区间
internal_metrics = {
    'accuracy': accuracy_score(y_internal_test, y_internal_pred_cv),
    'precision': precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0),
    'recall': recall_score(y_internal_test, y_internal_pred_cv, average='weighted'),
    'f1_score': f1_score(y_internal_test, y_internal_pred_cv, average='weighted'),
    'auc': roc_auc_score(y_internal_test, y_internal_proba_cv)
}

internal_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_internal_test, y_internal_proba_cv, roc_auc_score)
}

# 输出结果
print(f"Model with top 7 features: {selected_features}")
print("Best Parameters:", grid_voting.best_params_)

print("Training Set Metrics:")
for metric, value in train_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {train_ci[metric+'_ci'][0]:.4f}-{train_ci[metric+'_ci'][1]:.4f}")

print("Internal Validation Set Metrics:")
for metric, value in internal_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {internal_ci[metric+'_ci'][0]:.4f}-{internal_ci[metric+'_ci'][1]:.4f}")


Model with top 7 features: ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']
Best Parameters: {'dt__max_depth': None, 'lr__C': 10.0}
Training Set Metrics:
Accuracy: 0.8578 置信区间: 0.8088-0.9020
Precision: 0.8569 置信区间: 0.8049-0.9056
Recall: 0.8578 置信区间: 0.8088-0.9020
F1_score: 0.8573 置信区间: 0.8050-0.9036
Auc: 0.9050 置信区间: 0.8492-0.9507
Internal Validation Set Metrics:
Accuracy: 0.8636 置信区间: 0.7955-0.9318
Precision: 0.8636 置信区间: 0.7921-0.9318
Recall: 0.8636 置信区间: 0.7955-0.9318
F1_score: 0.8636 置信区间: 0.7893-0.9318
Auc: 0.9010 置信区间: 0.8254-0.9632


In [7]:
import pandas as pd

# 获取训练集的真实标签和预测概率
y_train_true = y_train.values
train_data = pd.DataFrame({
    'True_Label': y_train_true,
    'Predicted_Probability': y_train_proba_cv
})
train_data.to_csv(r'D:\desktop\B\结果\表格\VOT真实标签和预测概率(7)\train_labels_and_probabilities.csv', index=False)

# 获取内部验证集的真实标签和预测概率
y_internal_true = y_internal_test.values
internal_data = pd.DataFrame({
    'True_Label': y_internal_true,
    'Predicted_Probability': y_internal_proba_cv
})
internal_data.to_csv(r'D:\desktop\B\结果\表格\VOT真实标签和预测概率(7)\internal_validation_labels_and_probabilities.csv', index=False)

# 获取外部验证集的真实标签和预测概率
y_external_true = y_external_test.values
external_data = pd.DataFrame({
    'True_Label': y_external_true,
    'Predicted_Probability': y_external_proba_voting  # 这里应使用VotingClassifier模型的预测概率
})
external_data.to_csv(r'D:\desktop\B\结果\表格\VOT真实标签和预测概率(7)\external_validation_labels_and_probabilities.csv', index=False)

print("训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。")


训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。
